Growing up, my family generally ate healthy food, including fruits and vegetables. Indeed, my mother always made sure that my school lunch had not just a sandwich, but also a yellow vegetable and a green vegetable. I certainly knew that the other children didn’t eat many vegetables, but it took a good number of years until I realized just how few many of my friends ate.

I was reminded of that when reading a story in Axios several days ago, in which they quoted a study from the Centers for Disease Control and Prevention (CDC), claiming that half of young American children were eating less than one vegetable per day. I mean, we know that Americans don’t eat a lot of fresh vegetables, and we know that kids can be picky eaters, but … wow, could that possibly be true? And it true throughout the US?

This week, we’ll try to find out, by looking at the data from that CDC study, known as the National Survey of Children's Health (NSCH). The study looked at children 5 and younger, and asked all sorts of questions — not just about their eating habits, but also about such topics and allergies and speaking abilities. It also collected demographic information, including about race, which is important, but which we won’t look at here.

This week, I want you to download the data: https://www.census.gov/programs-surveys/nsch/data/datasets.html. In particular, here’s what I want you to do:

1. Download the topical data file, in SAS format, from:

https://www2.census.gov/programs-surveys/nsch/datasets/2021/nsch_2021_topical_SAS.zip. Turn it into a data frame. We're only interested in the following columns:

FIPSST

VEGETABLE

FRUIT

SUGARDRINK

2. Turn the FIPSST column into an integer, and make it the index.

3. What percentage of children had, on average, less than one vegetable per day during the week preceding the study?

4. What percentage of children had, on average, less than one vegetable per day and less than one fruit per day during the week preceding the study?

5. What percentage of children had, on average, less than one vegetable per day and less than one fruit per day and did have a sugary drink during the week preceding the study?

6. Download the FIPS state reference info, in CSV format, from https://www2.census.gov/geo/docs/reference/state.txt. Turn this into a data frame, with the STATE column as the index. The only other column we care about is STATE_NAME.

7. What percentage of children, per state, had, on average, less than one vegetaable per day during the week preceding the study?

The learning goals for this week are to work with SAS-formatted data, using boolean operators, and joining data from two data frames together.

In [1]:
# Import packages
import pandas as pd
import numpy as np

In [2]:
# Read the SAS data
filepath = '/Users/tomioredein/data_analyst/pandas_port/pandas_portfolio/pandas_port/Eating_well/nsch_2021e_topical.sas7bdat'
df = pd.read_sas(filepath)
df = df[["FIPSST","VEGETABLES","FRUIT","SUGARDRINK"]]
#

In [9]:
# Turn the FIPSST column into an integer, and make it the index.
df["FIPSST"] = df["FIPSST"].astype(np.int8)
# set the index
df = df.set_index("FIPSST")
df.head()


KeyError: 'FIPSST'

In [10]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 50892 entries, 48 to 28
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   VEGETABLES  18439 non-null  float64
 1   FRUIT       18523 non-null  float64
 2   SUGARDRINK  18551 non-null  float64
dtypes: float64(3)
memory usage: 1.2 MB


,VEGETABLES,FRUIT,SUGARDRINK
FIPSST,,,
48,3.0,2.0,1.0
2,NaN,NaN,NaN
40,NaN,NaN,NaN
26,2.0,2.0,2.0
22,2.0,3.0,2.0


In [19]:
# What percentage of children had, on average, less than one vegetable per day during the week preceding the study?

#  the first argument is our row selector, and our second argument is our column selector. We’re thus asking for the VEGETABLES column in df, but only where the value of VEGETABLES is less than 4.
perc_veg_child= (df.loc[df['VEGETABLES'] < 4, 'VEGETABLES'].count() / df["VEGETABLES"].count()) * 100
perc_veg_child



47.01448017788383

In [23]:
# what percentage of children had, on average, less than one vegetable per day and less than one fruit per day during the week preceding the study?


perc_veg_and_fruit_child = df.loc[(df['VEGETABLES'] < 4) & (df['FRUIT'] < 4),
       'VEGETABLES'].count() / df['VEGETABLES'].count()
perc_veg_and_fruit_child


0.2626498183198655

In [26]:
# What percentage of children had, on average, less than one vegetable per day and less than one fruit per day and did have a sugary drink during the week preceding the study?
df.loc[(df['VEGETABLES'] < 4) & (df['SUGARDRINK'] > 1) & (df['FRUIT'] < 4),
       'VEGETABLES'].count() / df['VEGETABLES'].count()


0.1590107923423179

In [27]:
# Download the FIPS state reference info, in CSV format, from https://www2.census.gov/geo/docs/reference/state.txt. Turn this into a data frame, with the STATE column as the index. The only other column we care about is STATE_NAME.

fips_url = 'https://www2.census.gov/geo/docs/reference/state.txt'
fips_df = pd.read_csv(fips_url, sep = '|', usecols=["STATE", "STATE_NAME"], index_col="STATE")

In [28]:
fips_df.head()

,STATE_NAME
STATE,
1,Alabama
2,Alaska
4,Arizona
5,Arkansas
6,California


In [29]:
joined_df = df.join(fips_df)
joined_df.head()

,VEGETABLES,FRUIT,SUGARDRINK,STATE_NAME
FIPSST,,,,
1,5.0,5.0,1.0,Alabama
1,NaN,NaN,NaN,Alabama
1,NaN,NaN,NaN,Alabama
1,3.0,3.0,1.0,Alabama
1,NaN,NaN,NaN,Alabama


In [30]:
# What percentage of children, per state, had, on average, less than one vegetaable per day during the week preceding the study?

# Logic We take all of the rows from the joined data frame
#
# Keep only those rows where VEGETABLES < 4
#
# Keep only the STATE_NAME and VEGETABLES columns
#
# For each value of STATE_NAME, count how many rows there are
#
# But we don’t want to know the raw number. We want to know the percentage of children eating so poorly. For that, we’ll first grab the VEGETABLES column (so that we get a series)

# Then we’ll divide that series by the total number of rows per state. This means that we’ll run a second “groupby”, this time without any filter:

(joined_df.loc[joined_df['VEGETABLES'] < 4,
              ['STATE_NAME','VEGETABLES']].groupby('STATE_NAME').count()['VEGETABLES'] / joined_df.groupby('STATE_NAME')['VEGETABLES'].count() * 100).sort_values()

STATE_NAME
Vermont                 30.000000
Maine                   33.501259
District of Columbia    35.218509
Minnesota               38.235294
New Hampshire           38.977636
Montana                 39.265537
Kansas                  40.860215
Oregon                  41.118421
Tennessee               41.964286
Alaska                  42.045455
California              42.151163
Colorado                42.574257
Ohio                    42.660550
Washington              42.896936
Massachusetts           42.948718
Wisconsin               43.283582
Maryland                43.971631
North Dakota            44.342508
Wyoming                 44.943820
Connecticut             45.405405
Iowa                    46.089385
North Carolina          46.710526
New Mexico              46.905537
South Carolina          47.040498
Michigan                47.222222
Missouri                47.321429
Pennsylvania            47.826087
Virginia                47.854785
West Virginia           48.245614
Geo